<a href="https://colab.research.google.com/github/raduga256/transformer_pegasus_text_paraphrase/blob/main/transformer_pegasus_text_paraphrase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## How to paraphrase text using transformers in Python

Notes and References:

PEGASUS is an acronym for Pre-training with Extracted Gap-sentences for Abstractive SUmmarization Sequence-to-sequence models
PEGASUS model used herein is from Huggingface's transformers library
PEGASUS model from Google Research
Read the original paper PEGASUS: Pre-training with Extracted Gap-sentences for Abstractive Summarization
Check out the book Transformers for Natural Language Processing

##Install Library

In [2]:
!pip install sentence-splitter
# splits the sentences and serve them as individual sentences for inputs

     |████████████████████████████████| 51kB 4.4MB/s 


In [3]:
!pip install transformers

     |████████████████████████████████| 2.1MB 9.1MB/s 
     |████████████████████████████████| 3.3MB 48.3MB/s 
     |████████████████████████████████| 870kB 26.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=d18c145f198e91f6bbe6cf7248d84e6e0783e5725f6be3aa541a72a81edc2e77
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [4]:
! pip install SentencePiece

     |████████████████████████████████| 1.2MB 9.1MB/s 


In [5]:
# https://huggingface.co/tuner007/pegasus_paraphrase

import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text, num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text], truncation=True, padding='longest', max_length=60, return_tensors='pt').to(torch_device)

  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

## Processing a single sentence

In [7]:
text = "In this video, I will be showing you how to build a stock price web application in Python using the Streamlit and yfinance library."

In [8]:
get_response(text, 5)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


['In this video, I will show you how to use the Streamlit and yfinance libraries to build a stock price web application.',
 'In this video, I will show you how to build a stock price web application in Python using the Streamlit and yfinance libraries.',
 'In this video, I will show you how to build a stock price web application using the Streamlit and yfinance libraries.',
 'In this video, I will show you how to use the Streamlit and yfinance libraries to build a stock price web application in Python.',
 'In this video, I will show you how to use the Streamlit and yfinance library to build a stock price web application.']

In [10]:
get_response(text, 1)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


['In this video, I will show you how to use the Streamlit and yfinance libraries to build a stock price web application.']

### Processing a paragraph of text

In [33]:
# paragraph of text
#context = "In this video, I will be showing you how to build a stock price web application in Python using the Streamlit and yfinance library. The app will be able to retrieve company information as well as the stock price data for S and P 500 companies. All of this in less than 50 lines of code."
context = "Maybe the abuse is not really that bad. Sure, he is violent but he can also be really nice and fun. I know she loves me but she just does not know how to show it. Maybe I am remembering things wrong."
print(context)

Maybe the abuse is not really that bad. Sure, he is violent but he can also be really nice and fun. I know she loves me but she just does not know how to show it. Maybe I am remembering things wrong.


In [34]:
 # Takes the input paragraph and splits it into a list of sentences
from sentence_splitter import SentenceSplitter, split_text_into_sentences

 splitter = SentenceSplitter(language='en')

 sentence_list = splitter.split(context)
 sentence_list 

['Maybe the abuse is not really that bad.',
 'Sure, he is violent but he can also be really nice and fun.',
 'I know she loves me but she just does not know how to show it.',
 'Maybe I am remembering things wrong.']

In [35]:
# Do a for loop to iterate through the list of sentences and paraphrase each sentence in the iteration
paraphrase = []

for i in sentence_list:
  a = get_response(i,1)
  paraphrase.append(a)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


In [36]:
# This is the paraphrased text
paraphrase

[['Maybe the abuse is not that bad.'],
 ['He can be violent but also fun.'],
 ["She doesn't know how to show her love for me."],
 ['Maybe I am forgetting something.']]

In [37]:
paraphrase2 = [' '.join(x) for x in paraphrase]
paraphrase2

['Maybe the abuse is not that bad.',
 'He can be violent but also fun.',
 "She doesn't know how to show her love for me.",
 'Maybe I am forgetting something.']

In [38]:
# Combines the above list into a paragrapgh
paraphrase3 = [' '.join(x for x in paraphrase2)]
paraphrased_text = str(paraphrase3).strip('[]').strip("'")
paraphrased_text

'"Maybe the abuse is not that bad. He can be violent but also fun. She doesn\'t know how to show her love for me. Maybe I am forgetting something."'

In [39]:
# Comparison of the original (context variable) ad the paraphrased version(paraphrase3)
print(context)
print(paraphrased_text)

Maybe the abuse is not really that bad. Sure, he is violent but he can also be really nice and fun. I know she loves me but she just does not know how to show it. Maybe I am remembering things wrong.
"Maybe the abuse is not that bad. He can be violent but also fun. She doesn't know how to show her love for me. Maybe I am forgetting something."
